<img src="https://imagedelivery.net/Dr98IMl5gQ9tPkFM5JRcng/3e5f6fbd-9bc6-4aa1-368e-e8bb1d6ca100/Ultra" alt="Image description" width="160" />

<br/>

# Introduction to Contextual AI Platform

The Contextual APIs provide a simple interface to our state-of-the-art Contextual Language Models (CLMs). Use this guide to learn the basics of how to create your first application programmatically. In this demo, we will be creating a RAG application for financial documents.

To run this notebook interactively, you can open it in Google Colab:

<a target="_blank" href="https://colab.research.google.com/github/ContextualAI/ContextualAI-Examples/blob/main/python/platform-api-sample.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup of Libraries, API, Dataset

To begin, you will need an API key to securely access the API. Please contact Contextual's sales team to get your API key.

In [ ]:
# Set up your API key
API_KEY = 'PASTE_YOUR_API_KEY_HERE'

In [ ]:
REQUEST_URL = 'https://api.contextual.ai/v1'

In [ ]:
# Import libraries
import requests
import json
from pathlib import Path
from typing import List, Optional, Dict
from IPython.display import display, JSON

In [ ]:
# Helper function to define headers for API calls
def get_headers(content_type: str = "application/json") -> Dict[str, str]:
    """
    Generate headers for API requests
    """
    return {
        "accept": "application/json",
        "Content-Type": content_type,
        "Authorization": f"Bearer {API_KEY}"
    }

## Step 1: Create your Datastore


You will need to first create a datastore for your Application using the  /datastores endpoint.  Each Application will have it's own datastore where the Application data will be stored securely.

In [ ]:
def create_datastore(name: str) -> Dict:
    """
    Create a new datastore using the Contextual AI API.

    Args:
        name: Name of the datastore

    Returns:
        Dict: The JSON response from the API
    """
    url = f"{REQUEST_URL}/datastores"

    payload = {"name": name}

    try:
        response = requests.post(url, headers=get_headers(), json=payload)
        response.raise_for_status()
        return response.json()

    except requests.exceptions.RequestException as e:
        print(f"Error creating datastore: {e}")
        raise

In [ ]:
try:
    result = create_datastore(name="new_demo_datastore") #TODO: Set a name for your datastore
    datastore_id = result['id']
    print(f"Datastore ID: {datastore_id}")
except Exception as e:
    print(f"Failed to create datastore: {e}")

## Step 2: Ingest documents into your datastore

You can now ingest documents into your Application's datastore using the /datastores endpoint. We've provided a sample document for you to use (Apple's 2023Q1 10-Q filing). You can also use your own documents here.

In [ ]:
# Function to download data from Google Drive
def download_google_drive_pdf(url: str, output_path: str):
   """
   Download a PDF from Google Drive using the file ID
   """
   response = requests.get(url)
   response.raise_for_status()
   with open(output_path, 'wb') as f:
       f.write(response.content)
   print(f"Downloaded {output_path}")

   file_path = Path(output_path).absolute()
   return str(file_path)

In [ ]:
# Download sample data from Google Drive
file_path = download_google_drive_pdf(
    "https://drive.google.com/uc?export=download&id=1k4C-o7mLEmthbeyhccNEjYrQb10UFOfT",
    "2023_Q1_APPLE.pdf"
)

In [ ]:
def ingest_file_to_datastore(datastore_id: str, file_path: str):
   """
   Upload a local file to Contextual AI datastore

   Args:
       datastore_id: ID of the target datastore
       file_path: Path to local file to upload
   """
   try:
       url = f"{REQUEST_URL}/datastores/{datastore_id}/documents"

       with open(file_path, 'rb') as f:
           response = requests.post(
               url,
               headers={
                   'accept': 'application/json',
                   'authorization': f"Bearer {API_KEY}"
               },
               files={'file': f}
           )
           response.raise_for_status()
           print(f"Successfully uploaded {file_path} to datastore {datastore_id}")
           return response.json()

   except requests.exceptions.RequestException as e:
       print(f"Upload failed: {str(e)}")
       raise

In [ ]:
ingest_file_to_datastore(datastore_id, file_path)

## Step 3: Create your Application

Now, you can use the `/applications` endpoint to create your Application and link it to the datastore.

In [ ]:
def create_application(
    name: str,
    description: str,
    system_guidelines: Optional[str] = None,
    datastore_ids: Optional[List[str]] = None
) -> Dict:
    """
    Create a new application in Contextual AI

    Args:
        name: Name of the application
        description: Description of the application
        system_guidelines: Guidelines for the system
        datastore_ids: Optional list of datastore IDs

    Returns:
        JSON response from the API
    """
    url = f"{REQUEST_URL}/applications"

    payload = {
        "name": name,
        "description": description,
        "system_guidelines": system_guidelines,
        "datastore_ids": datastore_ids or []
    }

    response = requests.post(
        url,
        headers=get_headers(),
        json=payload
    )
    response.raise_for_status()
    return response.json()

In [ ]:
# Sample system guideline
sample_guideline = '''
You are an AI assistant specialized in financial analysis and reporting. Your responses should be precise, accurate, and sourced exclusively from official financial documentation provided to you. Please follow these guidelines:

Data Analysis & Response Quality:
* Only use information explicitly stated in provided documentation (e.g., earnings releases, financial statements, investor presentations)
* Present comparative analyses using structured formats with tables and bullet points where appropriate
* Include specific period-over-period comparisons (quarter-over-quarter, year-over-year) when relevant
* Maintain consistency in numerical presentations (e.g., consistent units, decimal places)
* Flag any one-time items or special charges that impact comparability

Technical Accuracy:
* Use industry-standard financial terminology
* Define specialized acronyms on first use
* Never interchange distinct financial terms (e.g., revenue, profit, income, cash flow)
* Always include units with numerical values
* Pay attention to fiscal vs. calendar year distinctions
* Present monetary values with appropriate scale (millions/billions)

Response Format:
* Begin with a high-level summary of key findings when analyzing data
* Structure detailed analyses in clear, hierarchical formats
* Use markdown for lists, tables, and emphasized text
* Maintain a professional, analytical tone
* Present quantitative data in consistent formats (e.g., basis points for ratios)

Critical Guidelines:
* Do not make forward-looking projections unless directly quoted from source materials
* Avoid opinions, speculation, or assumptions
* If information is unavailable or irrelevant, clearly state this without additional commentary
* Answer questions directly, then stop
* Do not reference source document names or file types in responses
* Focus only on information that directly answers the query

For any analysis, provide comprehensive insights using all relevant available information while maintaining strict adherence to these guidelines and focusing on delivering clear, actionable information.
'''


In [ ]:
# Now let's try creating an application
try:
    app_response = create_application(
        name="Financial Analysis RAG v9",
        description="Application for analyzing the SEC filings of major companies",
        system_guidelines=sample_guideline,
        datastore_ids=[datastore_id]
    )
    # Store application ID for later use
    application_id = app_response['id']
    print(f"Application ID created: {application_id}")
except requests.exceptions.RequestException as e:
    print(f"Error creating application: {e}")

## Step 4: Query your Application

Finally, let's query our application to see if its working.

In [ ]:
def query_application(
   application_id: str,
   messages: List[Dict[str, str]],
   model_id: Optional[str] = None,
   conversation_id: Optional[str] = None,
   stream: bool = False
):
   """
   Query a Contextual AI application

   Args:
       application_id: Application ID of the application to query
       messages: List of message dictionaries with 'content' and 'role' keys
       model_id: Optional model ID for specific fine-tuned model
       conversation_id: Optional conversation ID for message history
       stream: Whether to stream the response
   """
   try:
       url = f"{REQUEST_URL}/applications/{application_id}/query"

       payload = {
           "messages": messages,
           "stream": stream
       }

       if model_id:
           payload["model_id"] = model_id
       if conversation_id:
           payload["conversation_id"] = conversation_id

       response = requests.post(
           url,
           headers=get_headers(),
           json=payload
       )
       response.raise_for_status()
       return response.json()

   except requests.exceptions.RequestException as e:
       print(f"Query failed: {str(e)}")
       raise

**Note:** It may take a few minutes for the document to be ingested and processed.

In [ ]:
result = query_application(
    application_id=application_id,
    messages=[{
        # Input your question here
        "content": "What is Apple Inc's revenue?",
        "role": "user",
    }]
)

print(result["message"]["content"])

In [ ]:
result = query_application(
    application_id=application_id,
    messages=[{
        # Input your question here
        "content": "What is Apple's net income in billions (not millions)?",
        "role": "user",
    }]
)

print(result["message"]["content"])

In [ ]:
result = query_application(
    application_id=application_id,
    messages=[{
        # Input your question here
        "content": "What is the value of Apple's non-current assets?",
        "role": "user",
    }]
)

print(result["message"]["content"])

## Next Steps

In this Notebook, we've created a simple RAG application in the finance domain. Our APIs offer many more capabilities than what is shown in this Notebook, including the ability to tune and evaluate your RAG system. You can learn more at [docs.contextual.ai](https://docs.contextual.ai/)